In [1]:
! pip install datasets transformers sacrebleu -qq

ERROR: keybert 0.2.0 has requirement numpy>=1.18.5, but you'll have numpy 1.18.0 which is incompatible.


In [2]:
ds_path='../../../data/dataset/lc-quad-wikidata-2021-10-01'

In [3]:
from datasets import load_dataset, load_metric, Dataset, DatasetDict
import pandas as pd
from tqdm import tqdm

In [4]:
raw_datasets = load_dataset("lc_quad")

Using custom data configuration default
Reusing dataset lc_quad (/home/shyaz/.cache/huggingface/datasets/lc_quad/default/2.0.0/139ee1f12aca006669dcc1f282ec02e126c69e7595453db443ab022643d54086)


In [5]:
raw_datasets["train"][1]

{'NNQT_question': 'What is {child of} of {husband} of {Ranavalona I} ?',
 'paraphrased_question': "What is the name of Ranavalona I's husband's child?",
 'question': "Who is the child of Ranavalona I's husband?",
 'sparql_dbpedia18': 'SELECT ?answer WHERE { ?statement1 <http://www.w3.org/1999/02/22-rdf-syntax-ns#subject> <http://wikidata.dbpedia.org/resource/Q169794> . ?statement1 <http://www.w3.org/1999/02/22-rdf-syntax-ns#predicate> <http://www.wikidata.org/entity/P26>. ?statement1 <http://www.w3.org/1999/02/22-rdf-syntax-ns#object> ?X . ?statement2 <http://www.w3.org/1999/02/22-rdf-syntax-ns#subject> ?X. ?statement2 <http://www.w3.org/1999/02/22-rdf-syntax-ns#predicate> <http://www.wikidata.org/entity/P22> . ?statement2 <http://www.w3.org/1999/02/22-rdf-syntax-ns#object> ?answer . }',
 'sparql_wikidata': 'SELECT ?answer WHERE { wd:Q169794 wdt:P26 ?X . ?X wdt:P22 ?answer}',
 'subgraph': 'left-subgraph',
 'template': 'C RCD xD . xD RDE ?E',
 'template_index': 8,
 'uid': 15554}

In [6]:
rep_dict = {"ASK": "ask", "WHERE": "where", "SELECT": "select", "{": "[","}": "]", }
rep_dict2 = {"{": "","}": "" }
def replace_all(text, dict):
    for i, j in dict.items(): text = text.replace(i, j)
    return text

In [7]:
# Trim + Replace LowerCase + Remove weirdly long Question

In [8]:
def prepare(ds):
    col = 'translation'
    df = pd.DataFrame(columns=[col])
    for d in tqdm(ds):
        try:
            qry = replace_all(d['sparql_wikidata'], rep_dict).strip()
            if len(d['question'])<250: 
                q = replace_all(d['question'], rep_dict2).strip()
                df = df.append({col: {'en':q, 'sparql': qry}}, ignore_index=True)
            if d['paraphrased_question']!=[] and len(d['paraphrased_question'])<250:
                q = replace_all(d['paraphrased_question'], rep_dict2).strip()
                df = df.append({col: {'en':q, 'sparql': qry}}, ignore_index=True)
        except: pass
    return df

In [9]:
df_test = prepare(raw_datasets["test"])

100%|██████████| 4781/4781 [00:17<00:00, 267.54it/s]


In [10]:
df_train = prepare(raw_datasets["train"])

 87%|████████▋ | 16779/19293 [00:59<00:12, 203.82it/s]

In [ ]:
df_test

,translation
0,{'en': 'Who is the country for head of state ...
1,{'en': 'What country is Mahmoud Abbas the head...
2,{'en': 'What was the population of Somalia in ...
3,"{'en': 'As of 2009, how many people lived in S..."
4,{'en': 'Which female actress is the voice over...
...,...
9542,{'en': 'Bernard Devoto received the Pulizer Pr...
9543,{'en': 'What is antonym of of spore print colo...
9544,{'en': 'What is antonym of of spore print colo...
9545,{'en': 'Tell me mixture whose name has the wor...


In [ ]:
df_train

,translation
0,{'en': 'What periodical literature does Delta ...
1,{'en': 'What is Delta Air Line's periodical li...
2,{'en': 'Who is the child of Ranavalona I's hus...
3,{'en': 'What is the name of Ranavalona I's hus...
4,{'en': 'Is it true Jeff_Bridges occupation Lan...
...,...
38508,{'en': 'IS THE Bubbling POINT OF THE METHANOL ...
38509,"{'en': 'What type of people live in Fresno, Ca..."
38510,{'en': 'What sort of individuals live in Fresn...
38511,{'en': 'For which film did Anil Kapoor win a S...


In [ ]:
df_all = pd.concat([df_train, df_test])

In [ ]:
# shuffling all
df_all  = df_all.sample(frac = 1)

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df_all, test_size=0.2)

In [ ]:
df_test

,translation
11602,{'en': 'When are Taxon products determined for...
31188,{'en': 'When does the twinned administrative b...
8430,{'en': 'Where is the workplace of Heinrich von...
9415,{'en': 'Name a historic region whose highest p...
7265,{'en': 'What is the surname of Keiko Matsuzaka...
...,...
35762,{'en': 'What prize does the champion of George...
31838,{'en': 'When is national occasion of comes fro...
27613,{'en': 'As described by Brockhaus and Efron En...
18354,{'en': 'Which streak color is strong arrangeme...


In [ ]:
ds_train = Dataset.from_pandas(df_train)
ds_test = Dataset.from_pandas(df_test)

In [ ]:
ds_test

Dataset({
    features: ['translation', '__index_level_0__'],
    num_rows: 9612
})

In [ ]:
mother_ds = DatasetDict({'train': ds_train, 'test':ds_test})

In [ ]:
mother_ds.save_to_disk(ds_path)